### IMPORTS

In [2]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [3]:
from Carga_Descarga import *
from CRM_Central import *

In [4]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### QUERIES

In [144]:
q_crm = '''SELECT vd.country_name AS Country,
       FORMAT_DATE('%Y-%m',vd.date) AS Month,
       vd.campaign_name AS Campaign,
       vd.coupon_type AS Type,
       SUM(CASE WHEN vd.payment_shopper = 0 THEN vd.coupon_used_amount_confirmed / ce.rate_us ELSE 0 END) AS Amount_Shopper,
       SUM(CASE WHEN vd.payment_shopper = 1 THEN vd.coupon_used_amount_confirmed / ce.rate_us ELSE 0 END) AS Amount
FROM `peya-growth-and-onboarding.automated_tables_reports.peya_vouchers_daily` AS vd
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON UPPER(vd.country_name) = UPPER(cn.country_name)
LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(vd.date,MONTH) = ce.currency_exchange_date
WHERE vd.date BETWEEN DATE('2021-03-01') AND DATE('2021-06-30')
GROUP BY 1,2,3,4'''

q_ord = '''SELECT cn.country_name AS Country,
       FORMAT_DATE('%Y-%m',o.registered_date) AS Month,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Confirmed,
       SUM(o.is_acquisition) AS Acq
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
WHERE o.registered_date BETWEEN DATE('2021-03-01') AND DATE('2021-06-30')
      AND cn.country_name != 'Colombia'
GROUP BY 1,2
ORDER BY 1,2'''

In [145]:
# Descargo la data
bq_crm = pd.io.gbq.read_gbq(q_crm, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████| 165850/165850 [00:27<00:00, 5930.75rows/s]


In [146]:
bq_ord = pd.io.gbq.read_gbq(q_ord, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 116.30rows/s]


In [152]:
# Copio las bases
crm = bq_crm.copy()
orders = bq_ord.copy()

### TRABAJO

In [153]:
# Doy formato a las columnas
cols_str = ['Type','Country','Month','Campaign']
cols_float = [i for i in crm.columns if i not in cols_str]
crm[cols_float] = crm[cols_float].astype(float)
crm[cols_str] = crm[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
crm['Campaign'] = crm['Campaign'].str.replace(' ', '')
# Coloco el IVA
crm = func_iva(crm)
# Marco las campañas a filtrar
crm['Filtrar'] = crm['Campaign'].apply(filtrar_cam)
crm = crm[crm['Filtrar'] == 'No'].copy()
# Marco el Budget
crm['Budget'] = crm['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
crm = crm[crm['Budget'] != 'NONE'].copy()

In [155]:
# Saco el IVA
used = 'Amount'
used_shopper = 'Amount_Shopper'
crm['Final'] = crm[used_shopper] / (1 + crm['IVA']) * crm['Extra'] + crm[used]

In [156]:
# Hago una PT
index = ['Country','Month','Type']
values = ['Final']
pt = crm.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()

In [157]:
# Hago una PT nueva
pt = pd.DataFrame(pt.pivot_table(index=['Country','Month'],columns=['Type'],values=['Final'],aggfunc='sum',fill_value=0).to_records())

In [158]:
# Doy formato a las columnas
cols_str = ['Country','Month']
cols_float = [i for i in orders.columns if i not in cols_str]
orders[cols_float] = orders[cols_float].astype(float)
orders[cols_str] = orders[cols_str].apply(lambda x: x.astype(str).str.upper())

In [159]:
# Hago un Merge final
final = orders.merge(pt,on=['Country','Month'],how='left')

### CARGA

In [162]:
carga(final,'1XV1UI2YTRUG_GLsASYIlVB9CA-fqbGLb7uCwFus_FOo','Crudo')

Carga Correcta!
